In [1]:
import evalml
from evalml import *
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *
from model_winner_select import *


from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

step = 1


In [3]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [4]:
load_from_disk = True
tec = TecAn(windows = 20, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(15, sourceDataGenerator, load_from_disk, 10)

In [5]:
trainX_raw, trainY_raw = online.load_train_data()
x, y = prepare_train_data(trainX_raw, trainY_raw, step)


print(Counter(y))

Counter({0: 20718, 1: 20718})


In [6]:

trainX_raw, trainY_raw = online.load_val_data("btcusd")
val_x, val_y = prepare_train_data(trainX_raw, trainY_raw, step)

print(Counter(val_y))

Counter({1: 1815, 0: 1815})


In [ ]:
%%time

from tpot import TPOTClassifier

model = TPOTClassifier(generations=30, 
                       population_size=10, 
                       scoring='precision_macro', 
                       verbosity=2, 
                       template='Transformer-Classifier',
                       random_state=1, 
                       n_jobs=-1)
# perform the search
model.fit(x, y)

Optimization Progress:   0%|          | 0/310 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6571772648808235

Generation 2 - Current best internal CV score: 0.6571772648808235

Generation 3 - Current best internal CV score: 0.689565683747482

Generation 4 - Current best internal CV score: 0.689565683747482

Generation 5 - Current best internal CV score: 0.6996965835563177

Generation 6 - Current best internal CV score: 0.6996965835563177

Generation 7 - Current best internal CV score: 0.6996965835563177

Generation 8 - Current best internal CV score: 0.6996965835563177

Generation 9 - Current best internal CV score: 0.6996965835563177

Generation 10 - Current best internal CV score: 0.7080747259476858

Generation 11 - Current best internal CV score: 0.7080747259476858

Generation 12 - Current best internal CV score: 0.7218309890393242

Generation 13 - Current best internal CV score: 0.7298820516643445

Generation 14 - Current best internal CV score: 0.7298820516643445

Generation 15 - Current best internal CV score: 0.72988205

In [ ]:
print(model.score(val_x, val_y))

In [ ]:
back = test_model(model, "btcusd", online, step, verbose = True)

back.report()

In [25]:
model.export('tpot_boston_pipeline.py')

In [ ]:
automl = AutoMLSearch(X_train=x, y_train=y, problem_type='binary', 
                      max_batches=4,
                      objective='auc',
                      #additional_objectives=['auc', 'f1', 'precision'],
                      #additional_objectives=[ 'precision'],
                      ensembling=False)
automl.search()

Generating pipelines to search over...
7 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 4 batches for a total of 23 pipelines. 
Allowed model families: extra_trees, linear_model, xgboost, catboost, random_forest, decision_tree



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean AUC: 0.500

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean AUC: 0.544
Decision Tree Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean AUC: 0.623
Random Forest Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean AUC: 0.648
CatBoost Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean AUC: 0.626
Extra Trees Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean AUC: 0.596
Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean AUC: 0.544


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-5ea463f04ef9>", line 7, in <module>
    automl.search()
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/evalml/automl/automl_search.py", line 820, in search
    if self._handle_keyboard_interrupt():
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/evalml/automl/automl_search.py", line 699, in _handle_keyboard_interrupt
    input(leading_char + "Do you really want to exit search (y/n)? ")
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 848, in raw_input
    return self._input_request(str(prompt),
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 892, in _i

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-5ea463f04ef9>", line 7, in <module>
    automl.search()
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/evalml/automl/automl_search.py", line 820, in search
    if self._handle_keyboard_interrupt():
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/evalml/automl/automl_search.py", line 699, in _handle_keyboard_interrupt
    input(leading_char + "Do you really want to exit search (y/n)? ")
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 848, in raw_input
    return self._input_request(str(prompt),
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 892, in _i

In [8]:
#automl.rankings

In [ ]:
pipeline = automl.best_pipeline

#pipeline.graph()

eval_data(pipeline, val_x, val_y)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 1391
Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-02da4a249cb0>", line 1, in <module>
    pipeline = automl.best_pipeline
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/evalml/automl/automl_search.py", line 1304, in best_pipeline
    raise PipelineNotFoundError(
evalml.exceptions.exceptions.PipelineNotFoundError: automl search must be run before selecting `best_pipeline`.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
Attribut

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-02da4a249cb0>", line 1, in <module>
    pipeline = automl.best_pipeline
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/evalml/automl/automl_search.py", line 1304, in best_pipeline
    raise PipelineNotFoundError(
evalml.exceptions.exceptions.PipelineNotFoundError: automl search must be run before selecting `best_pipeline`.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'PipelineNotFoundError' object has no attribute '_render_traceback_'

During handling of 

In [9]:
preds = pipeline.predict(val_x)

print(Counter(val_y))
print(Counter(preds))

ERROR! Session/line number was not unique in database. History logging moved to new session 1392


NameError: name 'pipeline' is not defined

In [ ]:
evalml.objectives.utils.get_core_objective_names()